In [16]:
!pip install kaggle

In [17]:
import os

os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
os.environ['KAGGLE_KEY'] = 'your_kaggle_api_key'

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

print("Kaggle API authentication successful.")

Kaggle API authentication successful.


In [18]:
!ls

mcc_codes.json			      sample_data	       transactions_data.csv
processed_transactions_2017_2019.csv  train_fraud_labels.json


In [19]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [20]:
!ls /root/.kaggle/

In [21]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [22]:
api.dataset_download_file("computingvictor/transactions-fraud-datasets", "transactions_data.csv")
api.dataset_download_file("computingvictor/transactions-fraud-datasets", "mcc_codes.json")
api.dataset_download_file("computingvictor/transactions-fraud-datasets", "train_fraud_labels.json")

Dataset URL: https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets
Dataset URL: https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets
Dataset URL: https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets


False

In [23]:
transactions_file = "transactions_data.csv"
mcc_codes_file = "mcc_codes.json"
fraud_labels_file = "train_fraud_labels.json"
processed_output_file = "processed_transactions_2017_2019.csv"

In [24]:
!ls

mcc_codes.json			      sample_data	       transactions_data.csv
processed_transactions_2017_2019.csv  train_fraud_labels.json


In [25]:
import pandas as pd
import json

In [26]:
with open(mcc_codes_file, "r") as f:
    mcc_codes = json.load(f)

with open(fraud_labels_file, "r") as f:
    fraud_labels = json.load(f)['target']

In [27]:
# Define date range for filtering
start_date = '2017-11-01'
end_date = '2019-11-01'

In [28]:
# Read transactions incrementally
chunksize = 10**6  # Adjust based on system memory
processed_chunks = []

for chunk in pd.read_csv(transactions_file, parse_dates=['date'], chunksize=chunksize):
    # Filter rows based on the date range
    filtered_chunk = chunk.loc[(chunk['date'] >= start_date) & (chunk['date'] <= end_date)].copy()

    # Add MCC descriptions
    filtered_chunk.loc[:, 'mcc_description'] = filtered_chunk['mcc'].astype(str).map(mcc_codes)

    # Add fraud labels
    filtered_chunk.loc[:, 'fraud_label'] = filtered_chunk['id'].astype(str).map(fraud_labels)

    # Append processed chunk
    processed_chunks.append(filtered_chunk)

KeyboardInterrupt: 

In [ ]:
# Combine all processed chunks into a single DataFrame
final_data = pd.concat(processed_chunks, ignore_index=True)
final_data.head()

In [ ]:
# Save the processed data
final_data.to_csv(processed_output_file, index=False)
print(f"Processed data saved to '{processed_output_file}'")

In [ ]:
# Cleaning data
cleaned_output_path = "cleaned_transactions_2017_2019.csv"
processed_data = pd.read_csv(processed_output_file, parse_dates=['date'])
processed_data.head(3)

In [ ]:
# 1. Drop unnecessary columns
columns_to_drop = ['errors']  # Add other unused columns here
processed_data = processed_data.drop(columns=columns_to_drop)
processed_data.head(3)

In [ ]:
# 2. Handle missing values
processed_data['merchant_state'] = processed_data['merchant_state'].fillna('Unknown')
processed_data['mcc_description'] = processed_data['mcc_description'].fillna('Unknown MCC')

In [ ]:
# 3. Normalize the `amount` column
processed_data['amount'] = processed_data['amount'].str.replace(r'[\$,]', '', regex=True).astype(float)
processed_data['amount'].head(3)

In [ ]:
# 4. Convert fraud labels to binary (handle missing or invalid values)
processed_data['fraud_label'] = processed_data['fraud_label'].fillna('no')  # Default to 'no' for NaN
processed_data['fraud_label'] = processed_data['fraud_label'].str.strip().str.lower().map({'yes': 1, 'no': 0})
processed_data['fraud_label'].head(3)

In [ ]:
# 6. Remove duplicates
processed_data = processed_data.drop_duplicates()
processed_data.head(3)

In [ ]:
# 7. Clean and validate zip codes
processed_data['zip'] = processed_data['zip'].fillna(0).astype(int)  # Replace NaN with 0 and convert to int
processed_data['zip'].head(3)

In [ ]:
# 8. Ensure consistent MCC descriptions
processed_data['mcc_description'] = processed_data['mcc_description'].str.strip()
processed_data['mcc_description'].head(3)

In [ ]:
# Save the cleaned dataset
processed_data.to_csv(cleaned_output_path, index=False)
print(f"Cleaned data saved to {cleaned_output_path}")
processed_data.head(3)

In [ ]:
#Data Analysis
df = processed_data
df.head(3)

In [ ]:
#Personal Spending calculations based on client id

psc = df[["client_id", "amount", "mcc_description"]]
psc.head(3)

In [ ]:
#Grouping all transactions by client id

client_amount = psc.groupby("client_id")
client_amount.get_group(0)

In [ ]:
#Computing highest spending category for each client

client_max_spending = client_amount.max()
client_max_spending

In [ ]:
#categorizing highest spending by mcc codes
client_max_plot = client_max_spending.groupby("mcc_description").count()
client_max_plot

In [ ]:
#Plot to visualize the count of client spending max amount on a mcc
client_max_plot.plot(kind="bar", figsize=(5,5), title='Highest spending categories for all clients', ylabel='Count')

In [ ]:
#Total amount spent by all client in 2 years on each mcc category

top_mcc = df[["mcc_description", "amount"]]
top_mcc.head(3)

In [ ]:
#Calculating top 10 mcc categories with most money spent

top_mcc_amt = top_mcc.groupby("mcc_description").sum()
top_mcc_plot = top_mcc_amt.sort_values("amount", ascending=False)
top_mcc_plot.head(10)

In [ ]:
#Plot for top 10 mcc codes for 2 years of transactions

top_mcc_plot.head(10).plot(kind="barh", figsize=(5,5), title="Top 10 MCC vs Total amount spent", xlabel="Total amount spent")

In [ ]:
#Fraud analysis for each transaction

fraud_ck = df[["use_chip","fraud_label"]]
fraud_ck.head(3)

In [ ]:
#Counting total number of transactions for each type

transaction_type = fraud_ck.groupby("use_chip").count()
transaction_type

In [ ]:
#Categorizing the fraud alerts based on the type of transaction (chip, online, swipe)

fr_aler = fraud_ck[fraud_ck.fraud_label ==1]
fr_count = fr_aler.groupby("use_chip").count()
fr_count

In [ ]:
#Calculating percentage fraud in each category
perc_fr = (fr_count.fraud_label/transaction_type.fraud_label)*100
perc_fr

In [ ]:
#Plot to identify the percentage of fraud
perc_fr.plot(kind='pie', autopct='%1.0f%%', figsize=(5,5), title='Fraud occurance by categories')

In [ ]:
#Data Export